# Overview

This notebook uses the 'TextFeaturesGenerator' class (from text_features) to convert textual data into qunatitaive data. 

For now, it creates a bag-of-words representation and a tf-idf representation. We will also add SVD/PCA of these matrices and a Word2Vec representation in the next few days.

Will update the TextFeaturesGenerator class on an ongoing basis and update the usage here.

In [1]:
from text_features import TextFeaturesGenerator
from project_helper import TweetData
import pandas as pd
import numpy as np

Reusing the TweetData class to get cleaned tweets.

In [2]:
tweet_data = TweetData()
tweet_data.clean_tweets.head()

,tweets
timestamp,
2019-10-02 23:41:51-05:00,democrats want to steal the election
2019-10-02 23:27:52-05:00,mississippi there is a very important election...
2019-10-02 23:27:52-05:00,he loves our military and supports our vets de...
2019-10-02 21:06:36-05:00,look at this photograph
2019-10-02 19:51:56-05:00,schiff house intel chairman got early account ...


Creating a 'TextFeaturesGenerator' instance which takes the tweets as an argument

In [3]:
feature_generator = TextFeaturesGenerator(tweet_data.clean_tweets.tweets)

'get_bow_matrix' creates the bag-of-words matrix

In [4]:
bow_mat = feature_generator.get_bow_matrix()

In [5]:
bow_mat.shape

(27960, 16781)

The shape of this matrix is 27.96K rows (same number as the tweets) and the columns are 16,781, which is equal to the unique number of words in the vocabulary.

In [6]:
bow_mat[:10,:10].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

As you can see, most of the values are zero which is why it is stored as a 'sparse-matrix'

Bag-of-words is simply a count of words in the tweet. A better representation is 'tf-idf'. The 'get_tfidf_matrix' creates

In [7]:
tfidf_mat = feature_generator.get_tfidf_matrix()
tfidf_mat.shape

(27960, 16781)

The matrices can be saved using the matrices function. You can either specify a 'folder' which will be created and both matrices stored in it, else will store in the working directory.

In [8]:
feature_generator.save_matrices()

The two matrices will be saved with the names "bow_mat.npz" and "tfidf_mat.npz"

You can also specify a folder and a suffix to the file names.

In [9]:
feature_generator.save_matrices(folder="matrices",suffix="_v2")

The files can be loaded using the following commands:

In [10]:
from scipy import sparse
bow_loaded = sparse.load_npz("bow_mat.npz")
tfidf_loaded = sparse.load_npz("tfidf_mat.npz")
print(bow_loaded.shape)
print(tfidf_loaded.shape)

(27960, 16781)
(27960, 16781)


## PCA (through SVD) of the matrices

You can get the SVD of the bow and tfidf matrices as well.

In [11]:
svd_bow_mat = feature_generator.get_svd_bow_mat()

In [12]:
svd_bow_mat.shape

(27960, 2)

By default, it gives back two components. You can changet that using the n_components argument.

In [13]:
svd_bow_mat = feature_generator.get_svd_bow_mat(n_components=100)

In [14]:
svd_bow_mat.shape

(27960, 100)

You can get the SVD of the tf-idf as well.

In [15]:
svd_tfidf_mat = feature_generator.get_svd_bow_mat(n_components=100)

In [16]:
svd_tfidf_mat.shape

(27960, 100)

These matrices can be saved as well.

In [17]:
feature_generator.save_matrices()

You can load them back using np.load

In [6]:
svd_loaded_mat = np.load('svd_tfidf_mat.npy')

In [7]:
svd_loaded_mat.shape

(27960, 100)

# Aggregagte SVD per day 

In [30]:
svd_df = pd.DataFrame(svd_loaded_mat)

In [31]:
svd_df['timestamp'] = tweet_data.clean_tweets.index
svd_df['date'] = svd_df.timestamp.dt.date

In [32]:
svd_df.head()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,timestamp,date
0,1.067114,0.077396,-0.803793,-0.205652,-0.505180,0.004843,0.039186,-0.049387,-0.095551,-0.017207,...,0.084714,0.092600,-0.132784,0.142841,0.065165,0.080188,0.070938,0.200888,2019-10-02 23:41:51-05:00,2019-10-02
1,2.398382,-2.635966,1.537750,0.957777,0.272032,-0.429117,-1.631507,0.552008,-1.720561,1.136831,...,0.189385,-0.185930,0.446110,0.174618,-0.213686,-0.321186,0.225457,0.057562,2019-10-02 23:27:52-05:00,2019-10-02
2,2.039926,-2.169685,2.635236,0.428123,-0.238948,-0.022691,-0.655132,-0.462281,-1.005658,1.190835,...,-0.261462,0.415466,0.034480,0.076168,-0.146568,-0.437547,0.395675,0.036453,2019-10-02 23:27:52-05:00,2019-10-02
3,0.116174,-0.057314,0.003153,0.019891,0.105173,-0.114806,-0.126431,-0.100204,-0.022856,-0.030551,...,0.030423,0.022345,-0.000702,0.019667,-0.096568,-0.054165,0.064102,-0.073907,2019-10-02 21:06:36-05:00,2019-10-02
4,0.473519,-0.025296,0.216236,0.710462,0.506245,0.856471,-0.055029,-0.015950,-0.015656,-0.262653,...,0.130805,0.003249,-0.051418,0.193854,-0.030467,0.001825,-0.097548,-0.090610,2019-10-02 19:51:56-05:00,2019-10-02


In [34]:
svd_df_daily = svd_df.groupby('date').agg(np.mean)

In [35]:
svd_df_daily.to_csv('svd_df_daily.csv',index=False)